# Hoja de trabajo 2

## Integrantes

- Sergio Orellana - 221122
- Rodrigo Mansilla - 22611


### Promt Utilizado

Responde a estas preguntas sobre ABM en epidemiología de forma clara y concisa: Cómo cambia la escalabilidad si los atributos de agentes son interdependientes vs ortogonales; Compensaciones entre transiciones de tiempo de espera (semi-Markov) y frecuencia/hazard (exponencial, sin memoria); Evidencia empírica de que el historial a nivel de agente mejora políticas reales frente a recuentos agregados; 4 cómo el modelado compositivo (p. ej., gráficos de estados jerárquicos y StockFlow.jl) mitiga la maldición de la dimensionalidad para comorbilidades.


### Ejercicio 1

- **¿Cómo cambiarían las ventajas de escalabilidad de los ABM si los atributos de los agentes exhibieran una fuerte interdependencia (por ejemplo, ingresos que modifican en gran medida los riesgos de enfermedades específicas de la edad) en comparación con el supuesto actual de heterogeneidad ortogonal?**

Si los atributos fueran fuertemente interdependientes, no se podría factorizar riesgos por atributo y se vería obligado a evaluar riesgos conjuntos/condicionales, mantener estados correlacionados y más historial por agente; eso eleva el costo por agente y la dificultad de calibración. En la práctica, se intensifica la maldición de la dimensionalidad: más dimensiones efectivas ⇒ más datos/cómputo y superficies de parámetros más complejas.

Aun así, incluso con interdependencias, un ABM  evita la explosión combinatoria típica de los modelos compartimentados cuando estratifico por múltiples atributos; los ABM retienen detalle individual sin multiplicar compartimentos, algo destacado en trabajos de ABM aplicados a COVID-19.

En términos de dinámica de sistemas (Sterman, 2000), la interdependencia introduce bucles de retroalimentación y efectos de path dependence que hacen el comportamiento más difícil de anticipar, pero también más realista frente a fenómenos epidemiológicos y socioeconómicos.

- **¿Qué compensaciones surgen al elegir entre transiciones con tiempos de espera (duraciones fijas) versus transiciones basadas en frecuencias (riesgos sin memoria) para modelar la progresión de enfermedades crónicas en los ABM?**

Con transiciones por frecuencia , se modelan salidas con una distribución exponencial (o geométrica en discreto), siendo esto  computacionalmente cómodo y compone bien procesos simultáneos,pues basta con evaluar riesgos en cada paso. Sin embargo, la propiedad sin memoria implica que la probabilidad de salir no depende del tiempo ya transcurrido en el estado, lo que distorsiona las distribuciones de permanencia en enfermedades crónicas, aparecen colas demasiado largas o cortas respecto a la evidencia empírica.

Con tiempos de espera (semi-Markov; duraciones fijas o con forma como log-normal), se capturan mejor sojourn times  es decir, el tiempo de permanencia en un estado antes de transitar a otro a costa de más contabilidad. Para esto hay que registrar el tiempo desde la entrada de cada agente y gestionar colas o eventos pendientes. Este razonamiento se expresa con la función de riesgo de supervivencia, que permite ajustar distribuciones de duración realistas. Es importante recordar que estos retrasos y la forma de las distriubuciones son importantes dado que generan dinámicas distintas. Con retrasos determinísticos aparecen oscilaciones más regulares, mientras que con distribuciones exponenciales el comportamiento es más difuso y difícil de anticipar


### Ejercicio 2

- **¿Qué evidencia empírica existe para validar que el seguimiento del historial a nivel de agente (por ejemplo, infecciones acumuladas) brinda mejores perspectivas políticas que los recuentos agregados de los modelos compartimentados en intervenciones de salud pública del mundo real?**

Veo evidencia en implementaciones reales donde el historial individual es esencial: OpenABM-Covid19 modela contactos y estados de intervención por persona (trazado manual/digital, aislamientos), y se aplicó operativamente en NHS England calibrando a datos NHS SitRep para monitoreo y respuesta; tales políticas dependen explícitamente de “quién-contactó-a-quién-y-cuándo”, algo que el conteo agregado no captura sin estallar en compartimentos.

Asimismo, Covasim documenta casos de uso con historias de infección/vacunación/aislamiento para evaluar paquetes de intervenciones y ajustar a datos observados, reforzando que el estado/historial por agente produce análisis de políticas más finos.

- **¿Cómo han abordado los recientes avances en el modelado compositivo (por ejemplo, los gráficos de estados jerárquicos) la “maldición de la dimensionalidad” a la hora de representar condiciones comórbidas en comparación con los enfoques tradicionales de stock y flujo?**

En mi modelado, represento comorbilidades componiendo sub-máquinas de estados jerárquicas/ortogonales (statecharts UML): {diabetes} × {EPOC} × {infección}, sincronizando solo donde hay dependencia. Esto reutiliza transiciones y evita enumerar la cruz completa de combinaciones como compartimentos separados.

En stock-and-flow, enfoques compositivos como StockFlow.jl permiten estratificar y recombinar módulos sin reescribir un ODE monolítico (Un único modelo enorme, con todas las ecuaciones entrelazadas, sin modularidad); la propuesta formaliza esa composición para mitigar la complejidad al añadir comorbilidades/estratos.

El trasfondo teórico sigue siendo la maldición de la dimensionalidad, que explica por qué la complejidad crece al aumentar atributos interdependientes.


## Referencias

- GeeksforGeeks. (2025, July 23). Poisson processes. GeeksforGeeks. https://www.geeksforgeeks.org/maths/poisson-processes/

- GeeksforGeeks. (2025a, April 23). Exponential distribution in NUMPY. GeeksforGeeks. https://www.geeksforgeeks.org/python/numpy-random-exponential-in-python

- GeeksforGeeks. (2025b, July 23). Hazard Function an overview. GeeksforGeeks. https://www.geeksforgeeks.org/data-science/hazard-function-an-overview/

- Kerr, C. C., Stuart, R. M., Mistry, D., Abeysuriya, R. G., Rosenfeld, K., Hart, G. R., Núñez, R. C., Cohen, J. A., Selvaraj, P., Hagedorn, B., George, L., Jastrzębski, M., Izzo, A. S., Fowler, G., Palmer, A., Delport, D., Scott, N., Kelly, S. L., Bennette, C. S., . . . Klein, D. J. (2021). Covasim: An agent-based model of COVID-19 dynamics and interventions. PLoS Computational Biology, 17(7), e1009149. https://doi.org/10.1371/journal.pcbi.1009149

- Baez, J., Li, X., Libkind, S., Osgood, N. D., & Patterson, E. (2023). Compositional Modeling with Stock and Flow Diagrams. Electronic Proceedings in Theoretical Computer Science, 380, 77–96. https://doi.org/10.4204/eptcs.380.5

- GeeksforGeeks. (2025c, July 23). KNearest Neighbors and Curse of Dimensionality. GeeksforGeeks. https://www.geeksforgeeks.org/machine-learning/k-nearest-neighbors-and-curse-of-dimensionality

- GeeksforGeeks. (2025a, April 8). State Machine Diagrams | Unified Modeling Language (UML). GeeksforGeeks. https://www.geeksforgeeks.org/system-design/unified-modeling-language-uml-state-diagrams/

- Railsback, S. F., & Grimm, V. (2012). *Agent-Based and Individual-Based Modeling: A Practical Introduction*. Princeton University Press.  

- Sterman, J. D. (2000). *Business Dynamics: Systems Thinking and Modeling for a Complex World*. McGraw-Hill.  
